# Proyecto 1 - MLOPS

**Integrantes:**

- Camilo Diaz Granados Nobman
- Luis Carlos Fernandez Vargas
- Daniel Alejandro Ruiz


Este proyecto busca evaluar la capacidad de crear un ambiente de desarrollo de machine learning
en el cual sea posible la ingesta, validación y transformación de datos, demostrando capacidad de
versionar código y ambiente de desarrollo.


Para el proyecto se utiliza una variante del conjunto de datos Tipo de Cubierta Forestal, de acuerdo con lo propuesto. Esto se puede utilizar **para entrenar un modelo que predice el tipo de cobertura forestal en función de variables cartográficas**.

## 1. Cargar el Dataset

Se procede a realizar la carga del dataset:

In [19]:
import os
import requests

# Cambiar la ruta del directorio a la nueva carpeta 'Data'
_data_root = '../data/covertype'
# Ruta al archivo de datos de entrenamiento
_data_filepath = os.path.join(_data_root, 'covertype_train.csv')
# Descargar datos
os.makedirs(_data_root, exist_ok=True)
if not os.path.isfile(_data_filepath):
    # URL del dataset
    url = 'https://docs.google.com/uc?export=download&confirm={{VALUE}}&id=1lVF1BCWLH4eXXV_YOJzjR7xZjj-wAGj9'
    r = requests.get(url, allow_redirects=True, stream=True)
    open(_data_filepath, 'wb').write(r.content)


## 2. Selección de Caracteristicas

Importamos las librerias necesarias para realizar la validación de los datos

In [20]:
import numpy as np
print(np.__version__)

1.26.4


In [21]:
import os
import requests
import pandas as pd
import tensorflow as tf
import tensorflow_data_validation as tfdv
from sklearn.model_selection import train_test_split
print('TF version:', tf.__version__)
print('TFDV version:', tfdv.version.__version__)

TF version: 2.16.2
TFDV version: 1.16.1


In [22]:
# Cargamos el dataset en un Dataframe desde la nueva ubicación
data_filepath = '../data/covertype/covertype_train.csv'
df = pd.read_csv(data_filepath, index_col=False)
df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2991,119,7,67,11,1015,233,234,133,1570,Commanche,C7202,1
1,2876,3,18,485,71,2495,192,202,144,1557,Commanche,C7757,1
2,3171,315,2,277,9,4374,213,237,162,1052,Rawah,C7745,0
3,3087,342,13,190,31,4774,193,221,166,752,Rawah,C7745,0
4,2835,158,10,212,41,3596,231,242,141,3280,Rawah,C4744,1


In [23]:
#Validamos la información de las caracteristicas
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116203 entries, 0 to 116202
Data columns (total 13 columns):
 #   Column                              Non-Null Count   Dtype 
---  ------                              --------------   ----- 
 0   Elevation                           116203 non-null  int64 
 1   Aspect                              116203 non-null  int64 
 2   Slope                               116203 non-null  int64 
 3   Horizontal_Distance_To_Hydrology    116203 non-null  int64 
 4   Vertical_Distance_To_Hydrology      116203 non-null  int64 
 5   Horizontal_Distance_To_Roadways     116203 non-null  int64 
 6   Hillshade_9am                       116203 non-null  int64 
 7   Hillshade_Noon                      116203 non-null  int64 
 8   Hillshade_3pm                       116203 non-null  int64 
 9   Horizontal_Distance_To_Fire_Points  116203 non-null  int64 
 10  Wilderness_Area                     116203 non-null  object
 11  Soil_Type                           116

Teniendo en cuenta que dos (2) de las caracteristicas son categoricas, creamos subconjuntos de las caracteristicas con el fin de calificar cuales tienen un mayor impacto en la predicción de la etiqueta "Cover_Type"

In [24]:
#Creamos subconjuntos
numeric_features = df.select_dtypes(include=[int, float])
categorical_features = df.select_dtypes(include=[object])

print("Características numéricas:")
print(numeric_features.info())

print("\nCaracterísticas categóricas:")
print(categorical_features.info())

Características numéricas:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116203 entries, 0 to 116202
Data columns (total 11 columns):
 #   Column                              Non-Null Count   Dtype
---  ------                              --------------   -----
 0   Elevation                           116203 non-null  int64
 1   Aspect                              116203 non-null  int64
 2   Slope                               116203 non-null  int64
 3   Horizontal_Distance_To_Hydrology    116203 non-null  int64
 4   Vertical_Distance_To_Hydrology      116203 non-null  int64
 5   Horizontal_Distance_To_Roadways     116203 non-null  int64
 6   Hillshade_9am                       116203 non-null  int64
 7   Hillshade_Noon                      116203 non-null  int64
 8   Hillshade_3pm                       116203 non-null  int64
 9   Horizontal_Distance_To_Fire_Points  116203 non-null  int64
 10  Cover_Type                          116203 non-null  int64
dtypes: int64(11)
memory usage

De acuerdo con las instrucciones del taller seleccionamos únicamente las caracteristicas númericas y separamos nuestra variable objetivo "Cover_Type".

In [25]:
# Definimos la variable objetivo y removemos Cover_Type de las columnas de entrada
y = numeric_features['Cover_Type']
X = numeric_features.drop('Cover_Type', axis=1, errors='ignore')

print("Dimensiones de X antes de la selección:", X.shape)

Dimensiones de X antes de la selección: (116203, 10)


Importamos las librerias para ejecutar la selección de las 8 mejores caracteristicas a través de "SelectKBest"

In [26]:
from sklearn.feature_selection import SelectKBest, f_classif

In [27]:
k = 8

# Crear el selector con la función de puntuación elegida
selector = SelectKBest(score_func=f_classif, k=k)

# Ajustar el selector y transformar X
X_selected = selector.fit_transform(X, y)

# Obtener los nombres de las columnas que han sido seleccionadas
selected_cols = X.columns[selector.get_support()]
print(f"Columnas seleccionadas (k={k}):\n", selected_cols.tolist())

Columnas seleccionadas (k=8):
 ['Elevation', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Horizontal_Distance_To_Fire_Points']


Se obtiene el resultado esperado según el taller, quitando de las caracteristicas seleccionadas a "Aspect" y "Hillshade3pm". Por lo que construimos un archivo CSV con los datos previamente seleccionados.

In [28]:
# Construir el DataFrame con las columnas seleccionadas y la variable target
df_selected = pd.DataFrame(X_selected, columns=selected_cols)
df_selected['Cover_Type'] = y.values

# Definir la ruta externa usando una ruta relativa
external_data_dir = os.path.abspath(os.path.join(os.getcwd(), '..', 'data'))
# Organiza las carpetas internas según necesites, en este caso:
output_dir = os.path.join(external_data_dir, 'selected_dataset')
os.makedirs(output_dir, exist_ok=True)

# Ruta completa donde se guardará el CSV
output_path = os.path.join(output_dir, 'selected.csv')
df_selected.to_csv(output_path, index=False)

## 3. Data Pipeline

Primero configuramos el contexto interactivo para ejecutar manualmente los componentes de canalización desde un cuaderno definiendo la ruta. 
La base de datos sqlite la mantenemos en el directorio Root de este Pipeline.

In [29]:
#Importamos las librerias necesarias
from tfx.components import CsvExampleGen
from tfx.components import ExampleValidator
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Transform

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from google.protobuf.json_format import MessageToDict

import pprint
pp = pprint.PrettyPrinter()

In [45]:
# Ubicación de la Metadata del Pipeline
_pipeline_root = './pipeline_root/'

# Directorio de los archivos de data "raw"
_data_root = '../data/selected_dataset'

# Ruta de la data de entrenamiento "raw" 
_data_filepath = os.path.join(_data_root, 'selected.csv')

In [46]:
# Inicializar el InteractiveContext con un archivo de sqlite local.
context = InteractiveContext(pipeline_root=_pipeline_root)

Validamos la ubicación de _data_root y con el fin de evitar que existan archivos adicionales en la carpeta, eliminamos lo que se encuentre para evitar conflictos en el contexto de exampleGen

In [47]:
os.listdir(_data_root)

['selected.csv', '.ipynb_checkpoints']

In [50]:
import os
import shutil

# Directorio que contiene el CSV y potencialmente otros archivos
data_dir = os.path.abspath(os.path.join(os.getcwd(), '..', 'data', 'selected_dataset'))

# Elimina la carpeta .ipynb_checkpoints si existe
checkpoints_path = os.path.join(data_dir, '.ipynb_checkpoints')
if os.path.exists(checkpoints_path) and os.path.isdir(checkpoints_path):
    shutil.rmtree(checkpoints_path)

In [51]:
os.listdir(_data_root)

['selected.csv']

### Generamos los Ejemplos

In [52]:
# Instanciar ExampleGen con el input CSV dataset
example_gen = CsvExampleGen(input_base=_data_root)

In [53]:
# Execute the component
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 2
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [54]:
# get the artifact object
artifact = example_gen.outputs['examples'].get()[0]

# print split names and uri
print(f'split names: {artifact.split_names}')
print(f'artifact uri: {artifact.uri}')

split names: ["train", "eval"]
artifact uri: ./pipeline_root/CsvExampleGen/examples/2


In [55]:
# Get the URI of the output artifact representing the training examples
train_uri = os.path.join(artifact.uri, 'Split-train')

# See the contents of the `train` folder
!ls {train_uri}

data_tfrecord-00000-of-00001.gz


In [56]:
# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

In [57]:
# Define a helper function to get individual examples
def get_records(dataset, num_records):
    '''Extracts records from the given dataset.
    Args:
        dataset (TFRecordDataset): dataset saved by ExampleGen
        num_records (int): number of records to preview
    '''
    
    # initialize an empty list
    records = []
    
    # Use the `take()` method to specify how many records to get
    for tfrecord in dataset.take(num_records):
        
        # Get the numpy property of the tensor
        serialized_example = tfrecord.numpy()
        
        # Initialize a `tf.train.Example()` to read the serialized data
        example = tf.train.Example()
        
        # Read the example data (output is a protocol buffer message)
        example.ParseFromString(serialized_example)
        
        # convert the protocol bufffer message to a Python dictionary
        example_dict = (MessageToDict(example))
        
        # append to the records list
        records.append(example_dict)
        
    return records

In [58]:
# Get 3 records from the dataset
sample_records = get_records(dataset, 3)

# Print the output
pp.pprint(sample_records)

[{'features': {'feature': {'Cover_Type': {'int64List': {'value': ['1']}},
                           'Elevation': {'int64List': {'value': ['2991']}},
                           'Hillshade_9am': {'int64List': {'value': ['233']}},
                           'Hillshade_Noon': {'int64List': {'value': ['234']}},
                           'Horizontal_Distance_To_Fire_Points': {'int64List': {'value': ['1570']}},
                           'Horizontal_Distance_To_Hydrology': {'int64List': {'value': ['67']}},
                           'Horizontal_Distance_To_Roadways': {'int64List': {'value': ['1015']}},
                           'Slope': {'int64List': {'value': ['7']}},
                           'Vertical_Distance_To_Hydrology': {'int64List': {'value': ['11']}}}}},
 {'features': {'feature': {'Cover_Type': {'int64List': {'value': ['1']}},
                           'Elevation': {'int64List': {'value': ['2876']}},
                           'Hillshade_9am': {'int64List': {'value': ['192']}},

2025-03-02 19:18:32.261450: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [59]:
# Verificar si ExampleGen generó datos
for artifact in example_gen.outputs['examples'].get():
    print(f"ExampleGen URI: {artifact.uri}")
    print("Archivos generados:", os.listdir(artifact.uri))

ExampleGen URI: ./pipeline_root/CsvExampleGen/examples/2
Archivos generados: ['Split-train', 'Split-eval']


## Generación de Estadisticas

In [60]:
# Instantiate StatisticsGen with the ExampleGen ingested dataset
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])

# Execute the component
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 3
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [61]:
print(statistics_gen)

StatisticsGen(spec: <tfx.types.standard_component_specs.StatisticsGenSpec object at 0x7fa7ad5a50f0>, executor_spec: <tfx.dsl.components.base.executor_spec.BeamExecutorSpec object at 0x7fa7ad5a5600>, driver_class: <class 'tfx.dsl.components.base.base_driver.BaseDriver'>, component_id: StatisticsGen, inputs: {'examples': OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)}, outputs: {'statistics': OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)})


In [62]:
print(statistics_gen.outputs['statistics'])

OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)


In [63]:
for example in example_gen.outputs['examples'].get():
    print(f"Example URI: {example.uri}")

Example URI: ./pipeline_root/CsvExampleGen/examples/2


In [64]:
# Show the output statistics
context.show(statistics_gen.outputs['statistics'])

## Inferir el Esquema

In [65]:
# Instantiate SchemaGen with the StatisticsGen ingested dataset
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    )

# Run the component
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 4
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [66]:
# Visualize the schema
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,-
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,-
'Hillshade_Noon',INT,required,,-
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,-
'Vertical_Distance_To_Hydrology',INT,required,,-


## Curando el Esquema

In [74]:
import tensorflow_data_validation as tfdv
 
# Cargar el esquema inferido previamente
schema_path = schema_gen.outputs['schema'].get()[0].uri + "/schema.pbtxt"
schema = tfdv.load_schema_text(schema_path)

Establecemos los rangos permitidos para ciertas características numéricas


In [75]:
feature_ranges = {
    "Hillshade_9am": (0, 255),
    "Hillshade_Noon": (0, 255),
    "Slope": (0, 90),
    "Cover_Type": (0, 6)
}

for feature, (min_val, max_val) in feature_ranges.items():
    feature_schema = tfdv.get_feature(schema, feature)
    feature_schema.int_domain.min = min_val
    feature_schema.int_domain.max = max_val

In [76]:
# Declarar 'Cover_Type' como categórica
tfdv.get_feature(schema, "Cover_Type").int_domain.is_categorical = True

In [77]:
# Guardar el esquema actualizado
tfdv.write_schema_text(schema, schema_path)

In [78]:
# Visualizar el esquema actualizado
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,min: 0; max: 6
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,min: 0; max: 90
'Vertical_Distance_To_Hydrology',INT,required,,-


Validamos que "Cover_Type" haya quedado como una variable categorica

In [79]:
cover_type_feature = tfdv.get_feature(schema, "Cover_Type")
print("¿Cover_Type es categórico?:", cover_type_feature.int_domain.is_categorical)

¿Cover_Type es categórico?: True


## Definimos los Entornos de Esquema

Previo a definir los entornos de esquema generamos un subconjunto de nuestros datos previamente filtrados en el que extraemos la etiqueta, simulando datos de servicio:

In [80]:
df_selected.head()

,Elevation,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Horizontal_Distance_To_Fire_Points,Cover_Type
0,2991,7,67,11,1015,233,234,1570,1
1,2876,18,485,71,2495,192,202,1557,1
2,3171,2,277,9,4374,213,237,1052,0
3,3087,13,190,31,4774,193,221,752,0
4,2835,10,212,41,3596,231,242,3280,1


In [81]:
# Eliminar la columna 'Cover_Type' para simular datos de servicio
df_serving = df_selected.drop(columns=["Cover_Type"])

In [82]:
df_serving.head()

,Elevation,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Horizontal_Distance_To_Fire_Points
0,2991,7,67,11,1015,233,234,1570
1,2876,18,485,71,2495,192,202,1557
2,3171,2,277,9,4374,213,237,1052
3,3087,13,190,31,4774,193,221,752
4,2835,10,212,41,3596,231,242,3280


In [83]:
# Creamos una carpeta para almacenar el subconjunto de datos de servicio
external_data_dir = os.path.abspath(os.path.join(os.getcwd(), '..', 'data'))
# Organiza las carpetas internas según necesites, en este caso:
output_dir = os.path.join(external_data_dir, 'serving_dataset')
os.makedirs(output_dir, exist_ok=True)

In [84]:
#Generamos un Subset de los datos
df_subset = df_serving.sample(frac=0.3, random_state=42)

In [85]:
#Guardar el nuevo subconjunto de datos de servicio
SERVING_DATA = "../data/serving_dataset/serving_data.csv"
df_subset.to_csv(SERVING_DATA, index=False)

In [87]:
#Cargar esquema previamente inferido
schema_path = schema_gen.outputs['schema'].get()[0].uri + "/schema.pbtxt"
schema = tfdv.load_schema_text(schema_path)

In [88]:
# Visualizar el esquema actualizado
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,min: 0; max: 6
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,min: 0; max: 90
'Vertical_Distance_To_Hydrology',INT,required,,-


In [89]:
#Definir entornos de esquema
schema.default_environment.append("TRAINING")
schema.default_environment.append("SERVING")

In [90]:
#Especificar que 'Cover_Type' NO está en el entorno de SERVING
tfdv.get_feature(schema, "Cover_Type").not_in_environment.append("SERVING")

In [91]:
#Generar estadísticas del conjunto de servicio
options = tfdv.StatsOptions(schema=schema, infer_type_from_schema=True)
serving_stats = tfdv.generate_statistics_from_csv(SERVING_DATA, stats_options=options)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


In [95]:
#Validar estadísticas con el Entorno de Esquma de Servicio
serving_anomalies_with_env = tfdv.validate_statistics(serving_stats, schema, environment="SERVING")

In [96]:
#Mostrar anomalías (debería estar limpio ahora)
tfdv.display_anomalies(serving_anomalies_with_env)

In [97]:
#Validar estadísticas con el Entorno de Esquema de Entrenamiento
serving_anomalies_with_env_Training = tfdv.validate_statistics(serving_stats, schema, environment="TRAINING")

In [98]:
#Mostrar anomalías
tfdv.display_anomalies(serving_anomalies_with_env_Training)

,Anomaly short description,Anomaly long description
Feature name,,
'Cover_Type',Column dropped,Column is completely missing


Se evidencia que al validar el subconjunto de datos de servicio contra el entorno de esquema de servicio no se evidencias anomalias. Mientras que al validarlos con el entorno de esquema de entrenamiento, identifica que esta ausente la columna de "Cover_Type" que es esperado.

In [99]:
#Guardar el esquema actualizado
tfdv.write_schema_text(schema, "updated_schema.pbtxt")

In [100]:
#Verificar que contiene los cambios
updated_schema = tfdv.load_schema_text("updated_schema.pbtxt")
print("Entornos disponibles:", updated_schema.default_environment)
cover_type_feature = tfdv.get_feature(updated_schema, "Cover_Type")
print("¿Cover_Type está en SERVING?", "SERVING" in cover_type_feature.not_in_environment)

Entornos disponibles: ['TRAINING', 'SERVING']
¿Cover_Type está en SERVING? True


## Generar Nuevas Estadisticas usando el esquema actualizado

In [103]:
from tfx.v1.components import ImportSchemaGen

In [104]:
#Importar el esquema actualizado desde un archivo
schema_gen = ImportSchemaGen(schema_file="updated_schema.pbtxt")

In [105]:
#Ejecutar el componente para registrar el esquema en los metadatos
context.run(schema_gen)

ExecutionResult(
    component_id: ImportSchemaGen
    execution_id: 5
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=ImportSchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [106]:
#Generar estadísticas con el esquema curado
statistics_gen2 = StatisticsGen(
    examples=example_gen.outputs['examples'],  # Proviene del ExampleGen definido previamente
    schema=schema_gen.outputs['schema']  # Usamos el esquema importado
)

In [107]:
#Ejecutar el componente
context.run(statistics_gen2)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 6
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [108]:
# Show the output statistics
context.show(statistics_gen2.outputs['statistics'])

## Comprobar anomalias

In [109]:
#Validar si hay anomalías en los datos
example_validator = ExampleValidator(
    statistics=statistics_gen2.outputs['statistics'],  # Usamos las estadísticas recién generadas
    schema=schema_gen.outputs['schema']  # Usamos el esquema curado
)

In [110]:
#Ejecutar el componente
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 7
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [111]:
#Mostrar anomalías detectadas
context.show(example_validator.outputs['anomalies'])

## Ingenieria de Caracteristicas

Se define primero el modulo de constantes

In [112]:
%%writefile forest_constants.py
# Archivo de constantes para el dataset Forest Cover Type

# Lista de características numéricas que se transformarán
NUMERIC_FEATURE_KEYS = [
    'Elevation',
    'Slope',
    'Horizontal_Distance_To_Hydrology',
    'Vertical_Distance_To_Hydrology',
    'Horizontal_Distance_To_Roadways',
    'Hillshade_9am',
    'Hillshade_Noon',
    'Horizontal_Distance_To_Fire_Points'
]

# La etiqueta a predecir
LABEL_KEY = 'Cover_Type'

# Características que queremos bucketizar (por ejemplo, 'Slope')
BUCKET_FEATURE_KEYS = ['Slope']

# Número de buckets para cada característica a bucketizar
FEATURE_BUCKET_COUNT = {'Slope': 5}

# Función de utilidad para renombrar las features transformadas
def transformed_name(key):
    return key + '_xf'


Writing forest_constants.py


## Funcion de Preprocesamiento

En el archivo forest_transform.py definiremos la función preprocessing_fn que aplica las transformaciones utilizando TensorFlow Transform

In [113]:
%%writefile forest_transform.py
import tensorflow as tf
import tensorflow_transform as tft
import forest_constants

# Extraer las constantes definidas
_NUMERIC_FEATURE_KEYS = forest_constants.NUMERIC_FEATURE_KEYS
_LABEL_KEY = forest_constants.LABEL_KEY
_BUCKET_FEATURE_KEYS = forest_constants.BUCKET_FEATURE_KEYS
_FEATURE_BUCKET_COUNT = forest_constants.FEATURE_BUCKET_COUNT
_transformed_name = forest_constants.transformed_name

def preprocessing_fn(inputs):
    """
    Función de callback para tf.transform que aplica transformaciones a las features.
    
    Args:
        inputs: Diccionario mapeando cada clave de feature a sus valores crudos.
    
    Returns:
        Diccionario mapeando las claves de las features transformadas a sus operaciones.
    """
    outputs = {}

    # 1. Escalar algunas features numéricas al rango [0, 1]
    for key in ['Elevation', 'Hillshade_9am', 'Hillshade_Noon']:
        outputs[_transformed_name(key)] = tft.scale_to_0_1(inputs[key])
    
    # 2. Escalar 'Slope' usando la normalización z-score
    outputs[_transformed_name('Slope')] = tft.scale_to_z_score(inputs['Slope'])
    
    # 3. Para las distancias, usamos scale_by_min_max
    for key in ['Horizontal_Distance_To_Hydrology', 
                'Vertical_Distance_To_Hydrology',
                'Horizontal_Distance_To_Roadways',
                'Horizontal_Distance_To_Fire_Points']:
        outputs[_transformed_name(key)] = tft.scale_by_min_max(inputs[key])
    
    # 4. Bucketización: además de normalizar, se crea una versión bucketizada de 'Slope'
    for key in _BUCKET_FEATURE_KEYS:
        outputs[_transformed_name(key) + '_bucketized'] = tft.bucketize(inputs[key],
                                                                        _FEATURE_BUCKET_COUNT[key])
    
    # 5. Convertir la etiqueta a un índice a través de un vocabulario.
    # Convertimos a string para obtener un vocabulario consistente.
    outputs[_transformed_name(_LABEL_KEY)] = tft.compute_and_apply_vocabulary(
                                                tf.as_string(inputs[_LABEL_KEY])
                                             )
    
    # 6. Ejemplo adicional: aplicar hashing a la etiqueta
    outputs[_transformed_name(_LABEL_KEY) + '_hashed'] = tft.hash_strings(
                                                            tf.as_string(inputs[_LABEL_KEY]),
                                                            hash_buckets=10
                                                         )
    
    return outputs

# Opcional: suprimir mensajes de advertencia de TensorFlow
tf.get_logger().setLevel('ERROR')


Writing forest_transform.py


## Transformar

Instanciamos el componente Transform de TFX

In [115]:
import tensorflow_transform as tft

In [116]:
# Instanciar el componente Transform usando el módulo de transformación
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath('forest_transform.py')
)

In [117]:
# Ejecutar el componente
context.run(transform)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying forest_transform.py -> build/lib
copying forest_constants.py -> build/lib
installing to /tmp/tmpwbsswp8k
running install
running install_lib
copying build/lib/forest_transform.py -> /tmp/tmpwbsswp8k
copying build/lib/forest_constants.py -> /tmp/tmpwbsswp8k
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmpwbsswp8k/tfx_user_code_Transform-0.0+0444d919caea8ec0b8cacb572089760d6cb28ee684b511d705509b774272493b

/usr/local/lib/python3.10/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(


Processing ./pipeline_root/_wheels/tfx_user_code_Transform-0.0+0444d919caea8ec0b8cacb572089760d6cb28ee684b511d705509b774272493b-py3-none-any.whl


Processing ./pipeline_root/_wheels/tfx_user_code_Transform-0.0+0444d919caea8ec0b8cacb572089760d6cb28ee684b511d705509b774272493b-py3-none-any.whl


Processing ./pipeline_root/_wheels/tfx_user_code_Transform-0.0+0444d919caea8ec0b8cacb572089760d6cb28ee684b511d705509b774272493b-py3-none-any.whl


ExecutionResult(
    component_id: Transform
    execution_id: 8
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [118]:
# Obtener la URI del grafo de transformación generado
transform_graph_uri = transform.outputs['transform_graph'].get()[0].uri
print("Transform Graph URI:", transform_graph_uri)

Transform Graph URI: ./pipeline_root/Transform/transform_graph/8


In [119]:
# Listar los subdirectorios bajo esa URI (opcional para depuración)
print("Subdirectorios en la URI del grafo de transformación:")
print(os.listdir(transform_graph_uri))

Subdirectorios en la URI del grafo de transformación:
['transformed_metadata', 'transform_fn', 'metadata']


In [120]:
# 3 Obtener la URI del artefacto de ejemplos transformados, asumiendo que se encuentra en el split 'Split-train'
transformed_examples_uri = transform.outputs['transformed_examples'].get()[0].uri
train_uri = os.path.join(transformed_examples_uri, 'Split-train')

In [122]:
#  Enlistar los nombres de los archivos TFRecord en el directorio 'Split-train'
tfrecord_filenames = [os.path.join(train_uri, name) for name in os.listdir(train_uri)]
print("Archivos TFRecord encontrados:")
pp.pprint(tfrecord_filenames)

Archivos TFRecord encontrados:
['./pipeline_root/Transform/transformed_examples/8/Split-train/transformed_examples-00000-of-00001.gz']


In [123]:
# Crear un dataset TFRecordDataset para leer los archivos (usando compresión GZIP)
transformed_dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

In [124]:
# Extraer 3 registros del dataset
sample_records_xf = get_records(transformed_dataset, 3)

2025-03-02 23:27:27.467511: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [125]:
# Imprimir los registros para inspeccionarlos
print("Ejemplos transformados:")
pp.pprint(sample_records_xf)

Ejemplos transformados:
[{'features': {'feature': {'Cover_Type_xf': {'int64List': {'value': ['0']}},
                           'Cover_Type_xf_hashed': {'int64List': {'value': ['9']}},
                           'Elevation_xf': {'floatList': {'value': [0.5686275]}},
                           'Hillshade_9am_xf': {'floatList': {'value': [0.9173228]}},
                           'Hillshade_Noon_xf': {'floatList': {'value': [0.9212598]}},
                           'Horizontal_Distance_To_Fire_Points_xf': {'floatList': {'value': [0.21902902]}},
                           'Horizontal_Distance_To_Hydrology_xf': {'floatList': {'value': [0.04886944]}},
                           'Horizontal_Distance_To_Roadways_xf': {'floatList': {'value': [0.14263631]}},
                           'Slope_xf': {'floatList': {'value': [-0.949044]}},
                           'Slope_xf_bucketized': {'int64List': {'value': ['0']}},
                           'Vertical_Distance_To_Hydrology_xf': {'floatList': {'

Al validar los ejemplos se confirma que las transformaciones se realizaron correctamente

# 3. Metadatos de Aprendizaje automático

In [126]:
# Importar los módulos de ML Metadata
from ml_metadata.proto import metadata_store_pb2
from ml_metadata.metadata_store import metadata_store

In [127]:
# --- Funciones auxiliares para inspeccionar artefactos ---

def get_records(dataset, num_records):
    '''Extrae registros del dataset TFRecord.
    Args:
        dataset (TFRecordDataset): dataset guardado (por ExampleGen)
        num_records (int): número de registros a extraer para mostrar
    '''
    records = []
    for tfrecord in dataset.take(num_records):
        serialized_example = tfrecord.numpy()
        example = tf.train.Example()
        example.ParseFromString(serialized_example)
        example_dict = MessageToDict(example)
        records.append(example_dict)
    return records

def display_types(types):
    # Función auxiliar para renderizar dataframes de tipos de artefacto/ejecución
    table = {'id': [], 'name': []}
    for a_type in types:
        table['id'].append(a_type.id)
        table['name'].append(a_type.name)
    return pd.DataFrame(data=table)

def display_artifacts(store, artifacts, base_dir):
    # Función auxiliar para renderizar dataframes de artefactos
    table = {'artifact id': [], 'type': [], 'uri': []}
    for a in artifacts:
        table['artifact id'].append(a.id)
        artifact_type = store.get_artifact_types_by_id([a.type_id])[0]
        table['type'].append(artifact_type.name)
        table['uri'].append(a.uri.replace(base_dir, './'))
    return pd.DataFrame(data=table)

def display_properties(store, node):
    # Función auxiliar para renderizar properties de artefactos y ejecuciones
    table = {'property': [], 'value': []}
    
    for k, v in node.properties.items():
        table['property'].append(k)
        table['value'].append(v.string_value if v.HasField('string_value') else v.int_value)
    
    for k, v in node.custom_properties.items():
        table['property'].append(k)
        table['value'].append(v.string_value if v.HasField('string_value') else v.int_value)
    
    return pd.DataFrame(data=table)

In [128]:
# Configuración de conexión para el almacén de metadatos basado en SQLite,
# usando la misma ruta que el InteractiveContext: ./pipeline_root/metadata.sqlite

metadata_connection_config = metadata_store_pb2.ConnectionConfig()
metadata_connection_config.sqlite.filename_uri = os.path.join(os.getcwd(), 'pipeline_root', 'metadata.sqlite')
metadata_connection_config.sqlite.connection_mode = (
    metadata_store_pb2.SqliteMetadataSourceConfig.READWRITE_OPENCREATE
)

In [130]:
# Inicializar el almacén de metadatos
store = metadata_store.MetadataStore(metadata_connection_config)

print("Conexión al almacén de metadatos establecida en:", 
      metadata_connection_config.sqlite.filename_uri)

Conexión al almacén de metadatos establecida en: /workspace/proyecto1/notebooks/pipeline_root/metadata.sqlite


## Acceso a Artefactos Almacenados

In [131]:
# --- Recuperar todos los tipos de artefactos almacenados ---

artifact_types = store.get_artifact_types()
df_types = display_types(artifact_types)
print("\nTipos de artefactos almacenados:")
print(df_types)


Tipos de artefactos almacenados:
   id               name
0  14           Examples
1  16  ExampleStatistics
2  18             Schema
3  21   ExampleAnomalies
4  23     TransformGraph
5  24     TransformCache


In [132]:
# --- Obtener la lista de artefactos de un tipo particular ---

# ver los artefactos del tipo 'Schema'.

schema_types = [atype for atype in artifact_types if atype.name == "Schema"]

if schema_types:
    schema_type_id = schema_types[0].id
    # Filtrar todos los artefactos cuyo type_id corresponde al tipo 'Schema'
    schema_artifacts = [artifact for artifact in store.get_artifacts() if artifact.type_id == schema_type_id]
    
    # Definir la ruta base (aquí se asume que la carpeta 'pipeline_root' se usa para los artefactos)
    pipeline_root = os.path.join(os.getcwd(), 'pipeline_root')
    df_schema_artifacts = display_artifacts(store, schema_artifacts, pipeline_root)
    
    print("\nArtefactos del tipo 'Schema' (deben incluir, por ejemplo, el esquema inferido y el actualizado):")
    # Mostramos al menos dos filas. Si hay más, se mostrará todo el DataFrame.
    print(df_schema_artifacts.head(2))
else:
    print("No se encontró ningún tipo de artefacto 'Schema'.")


Artefactos del tipo 'Schema' (deben incluir, por ejemplo, el esquema inferido y el actualizado):
   artifact id    type                                       uri
0            4  Schema        ./pipeline_root/SchemaGen/schema/4
1            5  Schema  ./pipeline_root/ImportSchemaGen/schema/5


In [133]:
# --- Obtener las propiedades de un artefacto en particular ---

if schema_artifacts:
    # Por ejemplo, tomamos el primer artefacto de tipo 'Schema'
    first_artifact = schema_artifacts[0]
    df_properties = display_properties(store, first_artifact)
    
    print("\nPropiedades del primer artefacto (Tipo 'Schema'):")
    print(df_properties)
else:
    print("No hay artefactos de tipo 'Schema' disponibles para mostrar propiedades.")


Propiedades del primer artefacto (Tipo 'Schema'):
             property                              value
0         tfx_version                             1.16.0
1                name  schema:2025-03-02T19:33:51.443265
2  producer_component                          SchemaGen


## Seguimiento de Artefactos

In [134]:
def get_primary_artifacts_for_target(store, target_artifact):
    """
    Dado un artifact de salida (por ejemplo, un TransformGraph), esta función
    recupera los artifacts de entrada que se utilizaron para generarlo.

    Args:
        store (MetadataStore): La conexión al almacén de metadatos.
        target_artifact: El artifact objetivo (por ejemplo, un TransformGraph).
    Returns:
        List of Artifact: Lista de artifacts primarios (input) asociados al execution
                          que generó target_artifact.
    """
    # 1. Obtener los eventos asociados al artifact objetivo.
    events = store.get_events_by_artifact_ids([target_artifact.id])
    
    # 2. Buscar el execution que produjo target_artifact.
    # Filtramos los eventos que tengan tipo OUTPUT.
    production_exec_ids = [e.execution_id for e in events 
                           if e.type == metadata_store_pb2.Event.OUTPUT]
    
    if not production_exec_ids:
        print("No se encontró un execution productor para el artifact id:", target_artifact.id)
        return []
    
    production_exec_id = production_exec_ids[0]
    
    # 3. Recuperar todos los eventos vinculados a este execution.
    events_for_execution = store.get_events_by_execution_ids([production_exec_id])
    
    # 4. Filtrar los eventos de tipo INPUT para extraer los artifact_ids asociados.
    input_artifact_ids = [e.artifact_id for e in events_for_execution 
                            if e.type == metadata_store_pb2.Event.INPUT]
    
    # 5. Recuperar los artifacts primarios usando sus IDs.
    primary_artifacts = store.get_artifacts_by_id(input_artifact_ids)
    
    return primary_artifacts

In [135]:
# 1. Recuperar todos los artifacts almacenados y filtrar los de tipo "TransformGraph"

# Primero, obtenemos todos los tipos de artefactos para saber qué nombres hay disponibles
artifact_types = store.get_artifact_types()
df_types = display_types(artifact_types)
print("\nTipos de artefactos registrados en MLMD:")
print(df_types)

# Ahora, obtenemos de todos los artifacts los que sean de tipo "TransformGraph"
all_artifacts = store.get_artifacts()
transform_graph_artifacts = [
    artifact 
    for artifact in all_artifacts 
    if store.get_artifact_types_by_id([artifact.type_id])[0].name == "TransformGraph"
]

if transform_graph_artifacts:
    # Seleccionamos el primer artifact de TransformGraph para el ejemplo
    target_artifact = transform_graph_artifacts[0]
    print("\nTransformGraph artifact seleccionado: ID =", target_artifact.id, ", URI =", target_artifact.uri)
    
    # ------------------------------------------------------------------------------
    # 2. Utilizar la función para obtener los artefactos primarios asociados a este artifact
    primary_artifacts = get_primary_artifacts_for_target(store, target_artifact)
    if primary_artifacts:
        print("\nArtefactos primarios asociados al TransformGraph:")
        # Suponiendo que la directorio base de artefactos es 'pipeline_root'
        pipeline_root = os.path.join(os.getcwd(), 'pipeline_root')
        df_primary = display_artifacts(store, primary_artifacts, pipeline_root)
        print(df_primary)
    else:
        print("No se encontraron artefactos primarios asociados al TransformGraph.")
    
    # ------------------------------------------------------------------------------
    # 3. Mostrar las propiedades del artifact TransformGraph seleccionado
    print("\nPropiedades del TransformGraph artifact seleccionado:")
    df_properties = display_properties(store, target_artifact)
    print(df_properties)
else:
    print("No se encontraron artefactos de tipo 'TransformGraph' en MLMD.")


Tipos de artefactos registrados en MLMD:
   id               name
0  14           Examples
1  16  ExampleStatistics
2  18             Schema
3  21   ExampleAnomalies
4  23     TransformGraph
5  24     TransformCache

TransformGraph artifact seleccionado: ID = 8 , URI = ./pipeline_root/Transform/transform_graph/8

Artefactos primarios asociados al TransformGraph:
   artifact id      type                                       uri
0            2  Examples  ./pipeline_root/CsvExampleGen/examples/2
1            5    Schema  ./pipeline_root/ImportSchemaGen/schema/5

Propiedades del TransformGraph artifact seleccionado:
             property                                       value
0         tfx_version                                      1.16.0
1  producer_component                                   Transform
2                name  transform_graph:2025-03-02T23:10:56.373881


## Obtener Artefactos Principales

In [136]:
all_artifacts = store.get_artifacts()
transform_graph_artifacts = [
    art for art in all_artifacts 
    if store.get_artifact_types_by_id([art.type_id])[0].name == "TransformGraph"
]
# Seleccionamos el primer TransformGraph para nuestro ejemplo
target_artifact = transform_graph_artifacts[0]
print("Artifact TransformGraph seleccionado: ID =", target_artifact.id, "URI =", target_artifact.uri)


Artifact TransformGraph seleccionado: ID = 8 URI = ./pipeline_root/Transform/transform_graph/8


In [137]:
# Obtener los eventos asociados al artifact (por su ID)
events = store.get_events_by_artifact_ids([target_artifact.id])
print("Eventos asociados al Artifact ID {}:".format(target_artifact.id))
for e in events:
    event_type = "OUTPUT" if e.type == metadata_store_pb2.Event.OUTPUT else "INPUT"
    print("  Execution ID:", e.execution_id, 
          " | Artifact ID:", e.artifact_id, 
          " | Tipo:", event_type)

Eventos asociados al Artifact ID 8:
  Execution ID: 8  | Artifact ID: 8  | Tipo: OUTPUT


In [139]:
# Recopilar execution IDs que producirían el artifact mediante eventos OUTPUT
execution_ids = { e.execution_id for e in events if e.type == metadata_store_pb2.Event.OUTPUT }
print("Execution ID(s) que produjeron el artifact:", execution_ids)

Execution ID(s) que produjeron el artifact: {8}


In [140]:
# Conjunto para almacenar los IDs de artifacts de entrada
input_artifact_ids = set()

# Por cada execution, obtener sus eventos y filtrar los INPUT
for exec_id in execution_ids:
    exec_events = store.get_events_by_execution_ids([exec_id])
    print("\nEventos para el Execution ID", exec_id)
    for ev in exec_events:
        ev_type = "OUTPUT" if ev.type == metadata_store_pb2.Event.OUTPUT else "INPUT"
        print("   Artifact ID:", ev.artifact_id, " | Tipo:", ev_type)
        if ev.type == metadata_store_pb2.Event.INPUT:
            input_artifact_ids.add(ev.artifact_id)

print("\nIDs de Artifacts de entrada encontrados:", input_artifact_ids)


Eventos para el Execution ID 8
   Artifact ID: 2  | Tipo: INPUT
   Artifact ID: 5  | Tipo: INPUT
   Artifact ID: 8  | Tipo: OUTPUT
   Artifact ID: 9  | Tipo: OUTPUT
   Artifact ID: 10  | Tipo: OUTPUT
   Artifact ID: 11  | Tipo: OUTPUT
   Artifact ID: 12  | Tipo: OUTPUT
   Artifact ID: 13  | Tipo: OUTPUT
   Artifact ID: 14  | Tipo: OUTPUT
   Artifact ID: 15  | Tipo: OUTPUT

IDs de Artifacts de entrada encontrados: {2, 5}
